In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf 
print([tf.__version__, tf.test.is_gpu_available()])

from sklearn.model_selection import KFold
from tensorboard.plugins.hparams import api as hp

['2.0.0', False]

In [2]:
help(hp.RealInterval)

Help on class RealInterval in module tensorboard.plugins.hparams.summary_v2:

class RealInterval(Domain)
 RealInterval(min_value=None, max_value=None)
 
 A domain that takes on all real values in a closed interval.
 
 Method resolution order:
 RealInterval
 Domain
 builtins.object
 
 Methods defined here:
 
 __init__(self, min_value=None, max_value=None)
 Create a `RealInterval`.
 
 Args:
 min_value: The lower bound (inclusive) of the interval.
 max_value: The upper bound (inclusive) of the interval.
 
 Raises:
 TypeError: If `min_value` or `max_value` is not an `float`.
 ValueError: If `min_value > max_value`.
 
 __repr__(self)
 Return repr(self).
 
 __str__(self)
 Return str(self).
 
 sample_uniform(self, rng=<module 'random' from '/local_disk0/pythonVirtualEnvDirs/virtualEnv-75baa417-53d6-4065-822b-3d1703868fd7/lib/python3.7/random.py'>)
 Sample a value from this domain uniformly at random.
 
 Args:
 rng: A `random.Random` interface; defaults to the `random` module
 itself.
 
 Raises:
 IndexError: If the domain is empty.
 
 update_hparam_info(self, hparam_info)
 Update an `HParamInfo` proto to include this domain.
 
 This should update the `type` field on the proto and exactly one of
 the `domain` variants on the proto.
 
 Args:
 hparam_info: An `api_pb2.HParamInfo` proto to modify.
 
 ----------------------------------------------------------------------
 Data descriptors defined here:
 
 dtype
 Data type of this domain: `float`, `int`, `str`, or `bool`.
 
 max_value
 
 min_value
 
 ----------------------------------------------------------------------
 Data and other attributes defined here:
 
 __abstractmethods__ = frozenset()
 
 ----------------------------------------------------------------------
 Data descriptors inherited from Domain:
 
 __dict__
 dictionary for instance variables (if defined)
 
 __weakref__
 list of weak references to the object (if defined)

In [3]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([128,256,512]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.4]))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [4]:
def train_model(hparams):
  input_shape = (28,28,1)
  model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=input_shape, padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(64, (5,5), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation='relu'),
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  print(model.summary())
  n_split = 4
  
  accuracy_list = []
  for train_index,test_index in KFold(n_split).split(x_data):
    x_train,x_test=x_data[train_index],x_data[test_index]
    y_train,y_test=y_data[train_index],y_data[test_index]
  
    model.fit(x_train, y_train,epochs=1, verbose=2)
  
    _, acc = model.evaluate(x_test,y_test, verbose=2)
    accuracy_list.append(acc)
    
  accuracy = np.array(accuracy_list).max() 
  return accuracy

In [5]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [6]:
# File location and type
file_location = "/FileStore/tables/train.csv"
test_location = "/FileStore/tables/test.csv"
file_type = "csv"

#customSchema = StructType()

# The applied options are for CSV files. For other file types, these will be ignored.
dataDF = sqlContext.read.format(file_type).options(delimiter=',', header='true', inferschema='true').load(file_location)
testDF = sqlContext.read.format(file_type).options(delimiter=',', header='true', inferschema='true').load(test_location)

x_data = dataDF.drop('label').toPandas()
y_data = dataDF.select('label').toPandas().values

x_data = (x_data.values/255.0).reshape(-1,28,28,1)

#(validDF, trainDF) = dataDF.randomSplit([15.0, 85.0])

#x_train = trainDF.drop('label').toPandas()
#y_train = trainDF.select('label').toPandas()

#x_valid = validDF.drop('label').toPandas()
#y_valid = validDF.select('label').toPandas()

#x_train = x_train.values.reshape(-1,28,28,1)/255.0
#x_valid = x_valid.values.reshape(-1,28,28,1)/255.0

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [7]:
session_num = 18

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.values):
    hparams = {
    HP_NUM_UNITS: num_units,
    HP_DROPOUT: dropout_rate
    }
    run_name = "run-%d" % session_num
    print('--- Starting trial: %s' % run_name)
    print({h.name: hparams[h] for h in hparams})
    run('logs/hparam_tuning/' + run_name, hparams)
    session_num += 1

--- Starting trial: run-18
{'num_units': 128, 'dropout': 0.4}
Model: "sequential_19"
_________________________________________________________________
Layer (type) Output Shape Param # 
=================================================================
conv2d_38 (Conv2D) (None, 28, 28, 32) 832 
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 14, 14, 32) 0 
_________________________________________________________________
conv2d_39 (Conv2D) (None, 14, 14, 64) 51264 
_________________________________________________________________
max_pooling2d_39 (MaxPooling (None, 7, 7, 64) 0 
_________________________________________________________________
flatten_19 (Flatten) (None, 3136) 0 
_________________________________________________________________
dense_38 (Dense) (None, 128) 401536 
_________________________________________________________________
dropout_19 (Dropout) (None, 128) 0 
_________________________________________________________________
dense_39 (Dense) (None, 10) 1290 
=================================================================
Total params: 454,922
Trainable params: 454,922
Non-trainable params: 0
_________________________________________________________________
None
Train on 31500 samples
31500/31500 - 50s - loss: 0.2427 - accuracy: 0.9263
10500/1 - 5s - loss: 0.0325 - accuracy: 0.9830
Train on 31500 samples
31500/31500 - 54s - loss: 0.0787 - accuracy: 0.9763
10500/1 - 5s - loss: 0.0210 - accuracy: 0.9887
Train on 31500 samples
31500/31500 - 51s - loss: 0.0558 - accuracy: 0.9841
10500/1 - 5s - loss: 0.0192 - accuracy: 0.9903
Train on 31500 samples
31500/31500 - 50s - loss: 0.0476 - accuracy: 0.9861
10500/1 - 5s - loss: 0.0116 - accuracy: 0.9933
--- Starting trial: run-19
{'num_units': 256, 'dropout': 0.4}
Model: "sequential_20"
_________________________________________________________________
Layer (type) Output Shape Param # 
=================================================================
conv2d_40 (Conv2D) (None, 28, 28, 32) 832 
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 14, 14, 32) 0 
_________________________________________________________________
conv2d_41 (Conv2D) (None, 14, 14, 64) 51264 
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 7, 7, 64) 0 
_________________________________________________________________
flatten_20 (Flatten) (None, 3136) 0 
_________________________________________________________________
dense_40 (Dense) (None, 256) 803072 
_________________________________________________________________
dropout_20 (Dropout) (None, 256) 0 
_________________________________________________________________
dense_41 (Dense) (None, 10) 2570 
=================================================================
Total params: 857,738
Trainable params: 857,738
Non-trainable params: 0
_________________________________________________________________
None
Train on 31500 samples
31500/31500 - 54s - loss: 0.1902 - accuracy: 0.9400
10500/1 - 6s - loss: 0.0330 - accuracy: 0.9822
Train on 31500 samples
31500/31500 - 52s - loss: 0.0621 - accuracy: 0.9815
10500/1 - 6s - loss: 0.0213 - accuracy: 0.9875
Train on 31500 samples
31500/31500 - 54s - loss: 0.0458 - accuracy: 0.9861
10500/1 - 6s - loss: 0.0189 - accuracy: 0.9920
Train on 31500 samples
31500/31500 - 52s - loss: 0.0354 - accuracy: 0.9885
10500/1 - 7s - loss: 0.0089 - accuracy: 0.9949
--- Starting trial: run-20
{'num_units': 512, 'dropout': 0.4}
Model: "sequential_21"
_________________________________________________________________
Layer (type) Output Shape Param # 
=================================================================
conv2d_42 (Conv2D) (None, 28, 28, 32) 832 
_________________________________________________________________
max_pooling2d_42 (MaxPooling (None, 14, 14, 32) 0 
_________________________________________________________________
conv2d_43 (Conv2D) (None, 14, 14, 64

In [8]:
prediction = model.predict(testDF.toPandas().astype('float32').values.reshape(-1,28,28,1)/255.0)

pre_lbl = np.argmax(prediction,axis=1)
output = pd.DataFrame(pre_lbl).reset_index()
output['index'] += 1
output = output.rename({'index': 'ImageId', 0: 'Label'}, axis = 1)
outputDF = spark.createDataFrame(output)

#save prediction as csv
(outputDF
 .coalesce(1)
 .write
 .format("csv")
 .options(delimiter=',', header="true")
 .save('FileStore/submission1'))